Note: you may need to restart the kernel to use updated packages.


In [2]:
import modin.pandas as pd
import re
import lxml.html


In [3]:
pd.read_gbq(f"""
select *
FROM`sotorrent-org.2020_12_31.PostType`

""", use_bqstorage_api=True)

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

To request implementation, send an email to feature_requests@modin.org.


,Id,Type
0,1,Question
1,2,Answer
2,3,Wiki
3,4,TagWikiExcerpt
4,5,TagWiki
5,6,ModeratorNomination
6,7,WikiPlaceholder
7,8,PrivilegeWiki


In [4]:
df_raw = pd.read_gbq (f"""

with qn as (
 select Id , AcceptedAnswerId, Title, Body as QuestionBody, Tags , ViewCount, AnswerCount,   CommentCount , Score, CreationDate
 FROM `sotorrent-org.2020_12_31.Posts` 
 where PostTypeId = 1
),
ans as (
 select Id , Body as AnswerBody
 FROM `sotorrent-org.2020_12_31.Posts` 
 where PostTypeId = 2
)


SELECT qn.*, ans.AnswerBody
From qn 
inner join ans 
on qn.AcceptedAnswerId = ans.Id

""", use_bqstorage_api=True)


In [5]:
df_raw.head()

,Id,AcceptedAnswerId,Title,QuestionBody,Tags,ViewCount,AnswerCount,CommentCount,Score,CreationDate,AnswerBody
0,22486469,22488014,Memory Mapping Large File Haskell,<p>I am experimenting with the Haskell mmap pa...,<haskell>,566,1,1,10,2014-03-18 17:18:08,<p>Looks like a typo. If I replace this:</p>\n...
1,20902775,20902933,How to check if auto-rotate screen setting is ...,<p>I think each android device has an abitily ...,<java><android>,11201,3,3,12,2014-01-03 11:37:35,<p>Hope this code snippet helps you out:-</p>\...
2,39613023,39623807,Understanding the FFT output,<p>I'm currently occupied in a practicum and m...,<java><fft>,277,1,0,-4,2016-09-21 09:46:43,<p>Computing a 512-point fourier transform aft...
3,2770630,2771563,PDO::fetchAll vs. PDO::fetch in a loop,<p>Just a quick question.</p>\n\n<p>Is there a...,<php><mysql><pdo><fetch>,86006,7,1,72,2010-05-05 04:31:40,<p>Little benchmark with 200k random records. ...
4,31725206,40180517,Unable to Flash eMMC from SD Card BeagleBone B...,<p>I am working on BeagleBone Black and Debian...,<debian><beagleboneblack>,31664,8,3,17,2015-07-30 13:30:39,"<p>Did you remember to remove the ""#"" at the b..."


In [6]:
df_raw.to_parquet("../data/df_raw",index=False)

In [7]:
df_raw = pd.read_parquet("../data/df_raw")

In [8]:
len(df_raw)

10597921

In [9]:
def strip_html(s:str):
    try:
        return str(lxml.html.fromstring(s).text_content())
    except:
        return ''

def parse_tags(content:str):
    return re.findall(r'<(.+?)>',content)


In [17]:
strip_html(f"""

<p>I was asked to create a singleton that will..</p>.	
""")

'I was asked to create a singleton that will...\t\n'

In [11]:
df = df_raw

In [13]:
df['QuestionBody'] =  df['QuestionBody'].apply(strip_html)

In [14]:
df['AnswerBody'] =  df['AnswerBody'].apply(strip_html)

In [15]:
df['Tags'] = df['Tags'].apply(parse_tags)

In [18]:
df.to_parquet("../data/df_processed/")